To covert to script
jupyter nbconvert somenotebook.ipynb --to script

In [1]:
# Import data
from __future__ import print_function
import os.path
import pandas as pd
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from _general import clean_text
import spacy
import subprocess
import sys
from gensim.models.word2vec import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
import multiprocessing
import numpy as np
from tqdm import tqdm
from annoy import AnnoyIndex
import json

In [2]:
# CONECTARSE A BASE DE DATOS 
# Could be cached
def connect_sheet(credentials):
    # If modifying these scopes, delete the file token.json.
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

    # Connect to database
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                credentials, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return creds


In [3]:
credential = connect_sheet("./archive/credentials.json")
credential

In [4]:
# CONSEGUIR DATOS DE BASE DE DATOS
def get_data(creds):

    # The ID and range of a sample spreadsheet.
    SAMPLE_SPREADSHEET_ID = '1Ge_nrrNF9VoVMysRiGm-Is53jSMh2OL_8cxTIimfpxE'
    bd_cotizaciones = 'BD_Cotizaciones_de_proveedores!A1:Q'
    planilla = 'Planilla!A1:G'

    # Call the Sheets API
    service = build('sheets', 'v4', credentials=creds) 
    sheet = service.spreadsheets()

    # Get data from database
    bd_cotizaciones_data = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=bd_cotizaciones).execute()
    bd_cotizaciones_data_values = bd_cotizaciones_data.get("values",[])

    planilla_data = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=planilla).execute()
    planilla_data_values = planilla_data.get("values",[])

    # Save as pandas dataframe
    df_bd = pd.DataFrame.from_records(bd_cotizaciones_data_values[1:],columns=bd_cotizaciones_data_values[0])
    df_planilla = pd.DataFrame.from_records(planilla_data_values[1:],columns=planilla_data_values[0])
    df_bd.to_csv("./data/productos_cotizados.csv", index=False)

    return df_bd, df_planilla

In [5]:
df_productos, df_planilla = get_data(credential)
df_productos

,Cotizacion,Producto Solicitado,Producto Ofrecido,U. Medida,Cantidad,Costo x Unidad,Costo x Unidad SIN IGV en Soles,Precio Venta x Unidad,Link,Observaciones,Habilitar y Deshabilitar,Adjunto,Adjunto Ficha Tecnica,Imagen Referencial,Ultima revisión,TRAZA,OC a proveedor
0,47630ddf50e6f0e4,5a76b72b,"MONITOR LG 21.5’’ (22MN430M-B), 1920 X 1080 (F...",Unidad,2,550.5,466.53,601.17,,,Deshabilitado,,,,,152e43f1,None
1,47630ddf50e6f0e4,3d56f01b,TECLADO INALAMBRICO + MOUSE INALAMBRICO GENIU...,Unidad,15,78.4,66.44,82.1,,,Deshabilitado,,,,,1ac5849a,None
2,47630ddfb5351092,3d56f01b,TECLADO INALAMBRICO + MOUSE INALAMBRICO TEROS...,Unidad,15,59.9,50.76,82.1,https://dasmitec.pe/productos/kit-inalambrico-...,,Deshabilitado,,,,,d79583f9,None
3,47630ddf05708cfd,3d56f01b,COMBO LOGITECH: TECLADO + MOUSE MK220 WIRELESS...,Unidad,15,77.5,77.5,82.1,https://www.memorykings.com.pe/producto/327993...,,Deshabilitado,,,,,aedc03a3,None
4,47630ddf05708cfd,5a76b72b,MONITOR 22” SAMSUNG LF22T350FHLXPE iPS FHD HDM...,Unidad,2,567.5,567.5,601.17,,,Habilitado,,,,,398d553b,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8373,3907425d,d97a6a43,Cámara Canon Powershot SX 70 HS.\nIncluye:\n(0...,Unidad,2,2999,2541.5254,2997,https://store.lenz.pe/products/camara-canon-po...,,Habilitado,,,Files/R2023-559/7a1fcc0c.Imagen Referencial.22...,,7a1fcc0c,None
8374,20a4f2e8,d97a6a43,Cámara digital Canon PowerShot SX70 HS\nConten...,Unidad,2,3199,2711.0169,3197,https://filmadorasperu.com/products/camara-dig...,,Habilitado,,,Files/R2023-559/ef0b4113.Imagen Referencial.22...,,ef0b4113,None
8375,a7534af2,05cf810c,CONDUCTOR ELECTRICO 1KV LIBRE HALOGENO 4MM2 NE...,Metro,30,4.5,4.5,7.5,,,Habilitado,Files/R2023-545/4f27df00.Adjunto.222940.pdf,,,,4f27df00,None
8376,dd86b2dd,d89f9fcf,ETIQUETA INKJET A4 003 PEGAFAN X 50 HOJAS,Paquete,10,32.3,27.3729,0,,** comprar rapido pq se acaba**,Habilitado,,,,,2495bc2c,None


In [6]:
df_planilla

,Cod Req,Categoria,Producto Solicitado,U. Medida,Cantidad,Observaciones,TRAZA
0,47630ddf,Cómputo,SOPORTE PARA LAPTOP 3M LX550,Unidad,0,,553486da
1,47630ddf,Cómputo,TECLADO INALAMBRICO + MOUSE INALAMBRICO,Unidad,0,,3d56f01b
2,47630ddf,Cómputo,MONITOR 22”,Unidad,0,,5a76b72b
3,47630ddf,Cómputo,SOPORTE PARA MONITOR,Unidad,7,,881845a2
4,cff1de24,Sillas,Sillas ejecutivas,Unidad,9,,d5c875c4
...,...,...,...,...,...,...,...
5068,4c2bcf9f,Misceláneos,Cámara CANON PowerShot SX70 HS,Unidad,2,,d97a6a43
5069,8b85c977,Eléctricos,ESTUFA ELÉCTRICA DE ACEITE SOLE 1500W modelo 3...,Unidad,2,,b3c450f7
5070,8b85c977,Misceláneos,ACEITE SINTÉTICO LE32H1,Galón,2,,029ba295
5071,8b85c977,Misceláneos,ETIQUETA INKJET A4 003 PEGAFAN X 50 HOJAS,Paquete,10,,d89f9fcf


In [7]:
# NORMALIZAR NOMBRES
def normalize(df,columna_producto):
    new_column ="Norm_" + columna_producto 
    df[new_column] = [clean_text(producto,True,True,True) for producto in df[columna_producto]]

    return df

In [8]:
df_productos = normalize(df_productos,"Producto Ofrecido")
df_productos

,Cotizacion,Producto Solicitado,Producto Ofrecido,U. Medida,Cantidad,Costo x Unidad,Costo x Unidad SIN IGV en Soles,Precio Venta x Unidad,Link,Observaciones,Habilitar y Deshabilitar,Adjunto,Adjunto Ficha Tecnica,Imagen Referencial,Ultima revisión,TRAZA,OC a proveedor,Norm_Producto Ofrecido
0,47630ddf50e6f0e4,5a76b72b,"MONITOR LG 21.5’’ (22MN430M-B), 1920 X 1080 (F...",Unidad,2,550.5,466.53,601.17,,,Deshabilitado,,,,,152e43f1,None,monitor lg 21 5 22mn430m b 1920 1080 fhd ips 2...
1,47630ddf50e6f0e4,3d56f01b,TECLADO INALAMBRICO + MOUSE INALAMBRICO GENIU...,Unidad,15,78.4,66.44,82.1,,,Deshabilitado,,,,,1ac5849a,None,teclado inalambrico mouse inalambrico genius k...
2,47630ddfb5351092,3d56f01b,TECLADO INALAMBRICO + MOUSE INALAMBRICO TEROS...,Unidad,15,59.9,50.76,82.1,https://dasmitec.pe/productos/kit-inalambrico-...,,Deshabilitado,,,,,d79583f9,None,teclado inalambrico mouse inalambrico teros 40...
3,47630ddf05708cfd,3d56f01b,COMBO LOGITECH: TECLADO + MOUSE MK220 WIRELESS...,Unidad,15,77.5,77.5,82.1,https://www.memorykings.com.pe/producto/327993...,,Deshabilitado,,,,,aedc03a3,None,combo logitech teclado mouse mk220 wireless us...
4,47630ddf05708cfd,5a76b72b,MONITOR 22” SAMSUNG LF22T350FHLXPE iPS FHD HDM...,Unidad,2,567.5,567.5,601.17,,,Habilitado,,,,,398d553b,None,monitor 22 samsung lf22t350fhlxpe ips fhd hdmi vg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8373,3907425d,d97a6a43,Cámara Canon Powershot SX 70 HS.\nIncluye:\n(0...,Unidad,2,2999,2541.5254,2997,https://store.lenz.pe/products/camara-canon-po...,,Habilitado,,,Files/R2023-559/7a1fcc0c.Imagen Referencial.22...,,7a1fcc0c,None,camara canon powershot sx 70 hs incluye 01 mem...
8374,20a4f2e8,d97a6a43,Cámara digital Canon PowerShot SX70 HS\nConten...,Unidad,2,3199,2711.0169,3197,https://filmadorasperu.com/products/camara-dig...,,Habilitado,,,Files/R2023-559/ef0b4113.Imagen Referencial.22...,,ef0b4113,None,camara digital canon powershot sx70 hscontenid...
8375,a7534af2,05cf810c,CONDUCTOR ELECTRICO 1KV LIBRE HALOGENO 4MM2 NE...,Metro,30,4.5,4.5,7.5,,,Habilitado,Files/R2023-545/4f27df00.Adjunto.222940.pdf,,,,4f27df00,None,conductor electrico 1kv libre halogeno 4mm2 ne...
8376,dd86b2dd,d89f9fcf,ETIQUETA INKJET A4 003 PEGAFAN X 50 HOJAS,Paquete,10,32.3,27.3729,0,,** comprar rapido pq se acaba**,Habilitado,,,,,2495bc2c,None,etiqueta inkjet a4 003 pegafan 50 hojas


In [9]:
df_planilla = normalize(df_planilla,"Categoria")
df_planilla = normalize(df_planilla,"Producto Solicitado")
df_planilla

,Cod Req,Categoria,Producto Solicitado,U. Medida,Cantidad,Observaciones,TRAZA,Norm_Categoria,Norm_Producto Solicitado
0,47630ddf,Cómputo,SOPORTE PARA LAPTOP 3M LX550,Unidad,0,,553486da,computo,soporte laptop 3m lx550
1,47630ddf,Cómputo,TECLADO INALAMBRICO + MOUSE INALAMBRICO,Unidad,0,,3d56f01b,computo,teclado inalambrico mouse inalambrico
2,47630ddf,Cómputo,MONITOR 22”,Unidad,0,,5a76b72b,computo,monitor 22
3,47630ddf,Cómputo,SOPORTE PARA MONITOR,Unidad,7,,881845a2,computo,soporte monitor
4,cff1de24,Sillas,Sillas ejecutivas,Unidad,9,,d5c875c4,sillas,sillas ejecutivas
...,...,...,...,...,...,...,...,...,...
5068,4c2bcf9f,Misceláneos,Cámara CANON PowerShot SX70 HS,Unidad,2,,d97a6a43,miscelaneos,camara canon powershot sx70 hs
5069,8b85c977,Eléctricos,ESTUFA ELÉCTRICA DE ACEITE SOLE 1500W modelo 3...,Unidad,2,,b3c450f7,electricos,estufa electrica aceite sole 1500w modelo 3120...
5070,8b85c977,Misceláneos,ACEITE SINTÉTICO LE32H1,Galón,2,,029ba295,miscelaneos,aceite sintetico le32h1
5071,8b85c977,Misceláneos,ETIQUETA INKJET A4 003 PEGAFAN X 50 HOJAS,Paquete,10,,d89f9fcf,miscelaneos,etiqueta inkjet a4 003 pegafan 50 hojas


In [10]:
# COMBINAR NOMBRES DFs
def combinar_bd(df1,df2,column1,column2):
    """
    Combines two pandas DFs

    ** df1 = pandas DF al que añadir valores
    ** df2 = pandas DF del que se sacaran valores
    ** column1,2 = Str. Nombre de la columna con el idx de connecion

    Lista a conseguir
    # Producto solicitado
    # Producto ofrecido
    # Categorias
    # Producto solicitado con categorias
    # Producto ofrecido con categorias
    # Producto solicitado con categoria y todos los prod ofrecidos
    """
    # Combinar dfs
    df = pd.merge(df1, df2,
                       how='left', left_on=column1, right_on=column2)
    
    # Make TRAIN_DATA
    TRAIN_DATA = []
    # Producto solicitado
    TRAIN_DATA = TRAIN_DATA + df["Norm_Producto Solicitado"].values.tolist() 
    # Producto ofrecido
    TRAIN_DATA = TRAIN_DATA + df["Norm_Producto Ofrecido"].values.tolist() 
    # Categorias
    TRAIN_DATA = TRAIN_DATA + df["Norm_Categoria"].values.tolist() 
    # Producto solicitado con categorias
    df["sol_categoria"] = df["Norm_Producto Solicitado"]+ " "+ df["Norm_Categoria"]
    TRAIN_DATA = TRAIN_DATA + df["sol_categoria"].values.tolist() 
    # Producto ofrecido con categorias
    df["ofr_categoria"] = df["Norm_Producto Ofrecido"]+ " "+ df["Norm_Categoria"]
    TRAIN_DATA = TRAIN_DATA + df["ofr_categoria"].values.tolist() 
    # Producto solicitado con categoria y todos los prod ofrecidos
    df["todo"] = df["Norm_Producto Solicitado"]+ " " + df["Norm_Categoria"] + " " +df["Norm_Producto Ofrecido"]
    TRAIN_DATA = TRAIN_DATA + df["todo"].values.tolist()  

    # Remove duplicates
    TRAIN_DATA = list(set(TRAIN_DATA))
    
    return TRAIN_DATA


In [11]:
TRAIN_DATA = combinar_bd(df_productos,df_planilla,"Producto Solicitado","TRAZA")
TRAIN_DATA

[nan,
 '',
 'balanza electronica altura peso 300kg 100g mantenimiento balanza electronica altura peso 300kg 100g',
 'hojas cartulina kimberly a4 220 grs blanco titanio',
 'ocion 2 hervidor oster 1 7 l blanco garantia 1 ano 2 dias cafeteria',
 'tacho vaiven color negro residuos generales 60 l limpieza tacho cosmos color negro residuos generales 60 l rey plast 2 dias',
 'papelera automatica clin metalizada 12 5l rey plasticos',
 'vela misionera 6',
 'escalera plataforma 8 peldanos raptor',
 'pano microfibra virutex 4 limpieza',
 'rack tv fijo inclinable 26 65 krearack negroinclinacion 15o',
 'televisor samsung led 50 uhd 4k smart tv un50au7090gxpe',
 '54 cera agua tekno 1 gl',
 'escalera telescopica 20 pasos',
 'caja 33cm 58 5cm 21cm 3 4 kg',
 '12 jabon liquido rosas galon daryza limpieza',
 'embudo aceite cuello flexible 2 gal',
 'esponja azul scotch brite paquete 6',
 'lavavajilla liquida sapolio galon 5 lt ',
 'limpiador polvo sapolio antibacterial azul frasco 450g limpieza pulidor po

In [12]:
# CREAR VECTORES
def create_vectors(data):
    """
    Crea gensim vectors
    ** data: list of words normalized and without stop words
    """

    # Split data
    texts = []
    for producto in data:
        if isinstance(producto,str) and len(producto)>0: 
            producto_split = producto.split(" ")
            palabras = []
            for word in producto_split:
                word = word.strip()
                
                if len(word) > 0:
                    palabras.append(word)
            texts.append(palabras)

    # Create wordVector
    cores = multiprocessing.cpu_count()
    w2v_model = Word2Vec(min_count=3,
                            window=2,
                            vector_size=500,
                            sample=6e-5,
                            alpha=0.03,
                            min_alpha=0.0007,
                            negative=20,
                            workers=cores-1)
    w2v_model.build_vocab(texts)
    w2v_model.train(texts, total_examples=w2v_model.corpus_count,epochs=30)

    # Save data
    w2v_model.wv.save_word2vec_format("data/vectores.txt")

    


In [13]:
create_vectors(TRAIN_DATA)

In [14]:
# COPIAR VECTORS EN MODELO DE SPACY
def load_word_vectors():

    model_name = "./model"

    nlp = spacy.blank("es-419")
    nlp.to_disk(model_name)


    subprocess.run([sys.executable,"-m","spacy",
                    "init", "vectors","es","data/vectores.txt",model_name])
    

In [15]:
load_word_vectors()

ℹ Creating blank nlp object for language 'es'


[2023-07-01 15:07:55,827] [INFO] Reading vectors from data/vectores.txt
13341it [00:00, 13910.03it/s]
[2023-07-01 15:07:56,837] [INFO] Loaded vectors from data/vectores.txt


✔ Successfully converted 13341 vectors
✔ Saved nlp object with vectors to output directory. You can now use
the path to it in your config as the 'vectors' setting in [initialize].
/Users/chavezmunoz.a/Documents/Dev/FacilInsumos/Historial/model


In [16]:
# USAR LISTA DE PRODUCTOS OFRECIDOS PARA CREAR INDICE
def create_index(df_productos):

    # Load model
    nlp = spacy.load("./model")

    # Get data from df
    train_list = df_productos[["Norm_Producto Ofrecido","TRAZA"]].values.tolist() 
    train_list = dict(zip(df_productos["TRAZA"], df_productos["Norm_Producto Ofrecido"]))

    # Create annoy object
    annoy_index = AnnoyIndex(500,'euclidean')

    # Make a dictionary for reference data
    reference = {}

    # For every produtct
    for ix,(traza,text) in tqdm(enumerate(train_list.items())):
        if text != None:

            # Convert to vector
            doc = nlp(text) 

            # Add to annoy index
            annoy_index.add_item(ix, doc.vector)

            # Add data to reference dictionary
            reference[ix] = (traza,text)
    annoy_index.build(10)

    # Save data
    annoy_index.save('./data/product_index.ann')

    with open("./data/reference_map.json", "w",encoding="utf-8") as f:
        json.dump(reference, f, indent = 4)
   
    return 

In [17]:
create_index(df_productos)

8344it [00:00, 8769.82it/s]


In [18]:
# Descargar script
!jupyter nbconvert --to python actualizar_datos.ipynb

[NbConvertApp] Converting notebook actualizar_datos.ipynb to python
[NbConvertApp] Writing 7968 bytes to actualizar_datos.py
